In [6]:
import os
from dotenv import load_dotenv
from pydantic import BaseModel
from pydantic_ai.models import KnownModelName
from pydantic_ai import Agent
from typing import cast
import logfire
from dataclasses import dataclass
from httpx import AsyncClient

load_dotenv()
logfire.configure(send_to_logfire='if-token-present')

In [7]:
@dataclass
class Deps:
    client: AsyncClient
    weather_api_key: str | None
    geo_api_key: str | None

In [ ]:
weather_agent = Agent(
    cast(KnownModelName, os.getenv(PYDANTIC_AI_MODEL))
)